# Improving ModSecurity WAF Using a Structured-Language Classifier


## For Computer Science B.Sc. Ariel University

By Maor Saadon, Shahar Zaidel, Eden Mor.

# Introdaction

In the ever-evolving landscape of cybersecurity, Web Application Firewalls (WAF) stand as critical defenders against malicious web requests. Traditionally, WAFs rely on predefined rule sets to identify and block potential threats. However, the complexity and sophistication of modern cyber attacks often outpace the capabilities of these rule-based systems, necessitating more dynamic and adaptive approaches. Machine learning (ML), particularly deep learning, has emerged as a promising solution, offering the potential to significantly enhance the detection and mitigation capabilities of WAFs through its ability to learn from data and identify complex patterns.

Our research explores the integration of machine learning techniques into the domain of web application security, focusing on the comparative analysis of various ML models in classifying web requests. We delve into models based on advanced text vectorization techniques, such as TF-IDF and Word2Vec, which transform web requests into numerical vectors that capture their semantic essence. These vectors are then utilized by machine learning algorithms, including the traditional k-Nearest Neighbors (kNN) algorithm, to distinguish between benign and malicious requests effectively.

Central to our investigation is the exploration of deep learning models in contrast to traditional ML models. Deep learning's capacity for learning hierarchical representations offers a nuanced approach to understanding and classifying web traffic. This leads us to our guiding research question: Does a machine learning model enhanced with deep learning techniques outperform traditional models in the context of Web Application Firewalls?

By examining the efficacy of deep learning models against their non-deep learning counterparts, our study aims to shed light on their relative performance and potential for improving the security mechanisms of WAFs. Through this comparative analysis, we aspire to contribute valuable insights into the advancement of cybersecurity methodologies, specifically in enhancing the resilience of web applications against the growing threat of cyber attacks.

## Install and import libraries

In [7]:
%pip install tensorflow
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import getopt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle as pkl
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import random

## Feature extraction functions


We will use the following feature extraction functions to extract features from the HTTP request:
1. calculate_alphanumeric_ratio - 
2. calculate_input_length - 
3. calculate_special_character_ratio - 


In [2]:
def calculate_alphanumeric_ratio(payload):
    alphanumeric_characters = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    alphanumeric_count = sum(1 for char in payload if char in alphanumeric_characters)
    payload_length = len(payload)
    input_length = max(payload_length, 1)  # Avoid division by zero if payload_length is 0
    alphanumeric_ratio = (alphanumeric_count / input_length) * 10
    return alphanumeric_ratio

def calculate_input_length(payload):
    input_length = len(payload)
    return input_length

def calculate_special_character_ratio(payload):
    alphanumeric_characters = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    special_count = sum(1 for char in payload if char not in alphanumeric_characters)
    payload_length = len(payload)
    input_length = max(payload_length, 1)  # Avoid division by zero if payload_length is 0
    special_ratio = (special_count / input_length) * 100
    return special_ratio


# Load and Procces the dataset 

The dataset is a CSV file with two columns: payload and label. The payload column contains the http request payload and the label column contains the label of the http request. The label is 1 if the request is malicious and 0 if the request is benign. 

In [3]:
# Load the dataset
df = pd.read_csv('CSIC.csv')
df = df.drop(['method','url','attack_feature'], axis=1)

# Shuffle the combined dataset to mix malicious and benign URLs
balanced_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print('Balanced Dataset Shape:', balanced_df.shape)
print('Balanced Dataset Distribution:', balanced_df['label'].value_counts())
print('Balanced Dataset Head (First 10 Rows):')
print(balanced_df.head(10))

Balanced Dataset Shape: (35602, 5)
Balanced Dataset Distribution: label
0    28150
1     7452
Name: count, dtype: int64
Balanced Dataset Head (First 10 Rows):
                                             payload  payload_len       alpha  \
0  modo=registro&login=bauer&password=P41DiD3M31t...          264  100.000000   
1  modo=insertar&precio=588&B1=Pasar+por+cajaparo...          152   66.355140   
2  modo=entrar&login=flaherty&pwd=j%2Baqu%E9s&rem...           65   89.285714   
3  id=%3F&nombre=Queso+Manchego&precio=85&cantida...           73   94.444444   
4                                                NaN            0    0.000000   
5                                                NaN            0    0.000000   
6                                  B2=Vaciar+carrito           17  100.000000   
7    id=%27%29%3Bwaitfor+delay+%270%3A0%3A15%27%3B--           47   64.285714   
8  modo=registro&login=schreife&password=EstOrnin...          257   98.675497   
9  id=3&nombre=Queso+Manchego&p

## Start testing with the options

We'll split the truncated dataset into a training set and a test set (80% training, 20% test) and use the training set to train the model and the test set to evaluate the model.

# W2V Vectorizer preprocessing

In [ ]:
#put your code

# CountVectorizer

In [19]:
# Fill NaN values with an empty string
def custom_tokenizer(text):
    return text.split('/')

# Fill NaN values with an empty string
balanced_df['payload'] = balanced_df['payload'].fillna("")

# Initialize the CountVectorizer with custom tokenizer
vectorizer = CountVectorizer(max_features=5000, max_df=0.85, tokenizer=custom_tokenizer, stop_words=['https', 'localhost:8080']) # , max_df=0.85, tokenizer=custom_tokenizer, stop_words=['https', 'localhost:8080']

# Fit and transform the 'payload' column
X = vectorizer.fit_transform(balanced_df['payload'])

# Convert the result to an array if needed
X_array = X.toarray()

# Optionally, convert to DataFrame for better readability
feature_names = vectorizer.get_feature_names_out()
X_df = pd.DataFrame(X_array, columns=feature_names)

# Apply feature extraction
print('Feature extraction...')
features = balanced_df['payload'].apply(lambda x: pd.Series({
    'payload_len': calculate_input_length(str(x)),
    'alpha': calculate_alphanumeric_ratio(str(x)),
    'non_alpha': calculate_special_character_ratio(str(x)),
    }))


# Concatenate original DF with features
X_df = pd.concat([X_df, features], axis=1)
print('Balanced Dataset Shape:', X_df.shape)
print('Balanced Dataset:')
print(X_df)

# Define y
y = balanced_df['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)


C:\Users\shaha\PycharmProjects\CMD_final_project\venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Feature extraction...
Balanced Dataset Shape: (35602, 5003)
Balanced Dataset:
          b2=%22%3e%3c%21--%23exec+cmd%3d%22dir+%22--%3e%3c  \
0      0                                                  0   
1      0                                                  0   
2      0                                                  0   
3      0                                                  0   
4      1                                                  0   
...   ..                                                ...   
35597  1                                                  0   
35598  1                                                  0   
35599  1                                                  0   
35600  0                                                  0   
35601  1                                                  0   

       b2=%22+and+%221%22%3d%221  b2=%2500  b2=%2520  b2=%252b  \
0                              0         0         0         0   
1                              0 

## K-Nearest Neighbors model

In [20]:

# Create an instance of kNN classifier
# the number of neighbors (k= 350) 
knn_model = KNeighborsClassifier(n_neighbors=350) 

# Train the kNN model (x_train-train_features,y_train-train_labels)
knn_model.fit(X_train, y_train)
print("K-Nearest Neighbors training is complete,\n predictions: ")

# Predict on the test set
predictions = knn_model.predict(X_test)

# #Evaluate the model
# accuracy = accuracy_score(y_test, predictions)
# CMatrix = confusion_matrix(y_test, predictions)

# # Print all evaluation metrics
# print("Accuracy:", accuracy,"Confusion Matrix:", CMatrix ,"\n","Classification Report:\n", classification_report(y_test, predictions))

# # Extract recall, precision, and F1-score from the classification report
# classification_dict = classification_report(y_test, predictions, output_dict=True)
# recall = classification_dict['1']['recall']  # Recall for the positive class (malicious)
# precision = classification_dict['1']['precision']  # Precision for the positive class (malicious)
# f1_score = classification_dict['1']['f1-score']  # F1-score for the positive class (malicious)

# # Dump the model to disk
# with open('knn_model_counter.pkl', 'wb') as f:
#     pkl.dump(knn_model, f)

# print('K-Nearest Neighbors model saved to disk.')

# print("Recall:", recall)
# print("Precision:", precision)
# print("F1 score:", f1_score)


#y_test = y_test.values.ravel()

print('[+] \t Classification accuracy : ', "{:.2f}".format(metrics.accuracy_score(y_test, predictions) * 100), '%\n')

print('[+] \t Percentage of Anomaly requests in test set : ', "{:.2f}".format(y_test.mean() * 100), '%\n')
print('[+] \t Percentage of Normal requests in test set : ', "{:.2f}".format((1 - y_test.mean()) * 100), '%\n')

confusion = metrics.confusion_matrix(y_test, predictions)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

print('[+] \t TP : ', TP, 'from ', (TP + TN + FP + FN))
print('    \t TN : ', TN, 'from ', (TP + TN + FP + FN))
print('    \t FP : ', FP, 'from ', (TP + TN + FP + FN))
print('    \t FN : ', FN, 'from ', (TP + TN + FP + FN))

print('\n[+] \t Metrics : ')
print('\t[-]  Accuracy Score (train_test_split): ', "{:.2f}".format(metrics.accuracy_score(y_test, predictions) * 100), '%')
print('\t[-]  Accuracy Score (k-fold): ',
      "{:.2f}".format(cross_val_score(knn_model, X_train, y_train, cv=100, scoring='accuracy').mean() * 100), '%')

print('\t[-]  Classification Error : ', "{:.2f}".format((1 - metrics.accuracy_score(y_test, predictions)) * 100), '%')
print('\t[-]  Recall : ', "{:.2f}".format(metrics.recall_score(y_test, predictions) * 100), '%')
specificity = TN / (TN + FP)
print('\t[-]  Specificity : ', "{:.2f}".format(specificity * 100), '%')
false_positive_rate = FP / float(TN + FP)
print('\t[-]  False Positive Rate : ', "{:.2f}".format(false_positive_rate * 100), '%')
precision = TP / float(TP + FP)
print('\t[-]  Precision : ', "{:.2f}".format(precision * 100), '%')


K-Nearest Neighbors training is complete,
 predictions: 
[+] 	 Classification accuracy :  93.26 %

[+] 	 Percentage of Anomaly requests in test set :  20.95 %

[+] 	 Percentage of Normal requests in test set :  79.05 %

[+] 	 TP :  1045 from  7121
    	 TN :  5596 from  7121
    	 FP :  33 from  7121
    	 FN :  447 from  7121

[+] 	 Metrics : 
	[-]  Accuracy Score (train_test_split):  93.26 %
	[-]  Accuracy Score (k-fold):  93.08 %
	[-]  Classification Error :  6.74 %
	[-]  Recall :  70.04 %
	[-]  Specificity :  99.41 %
	[-]  False Positive Rate :  0.59 %
	[-]  Precision :  96.94 %


## TensorFlow neural network, with feature extraction

A deep learning model using a Dense Neural Network (DNN), also known as a Fully Connected Network. This type of neural network architecture is characterized by its layers of neurons where each neuron in one layer is connected to all neurons in the next layer.

In [18]:
from sklearn.preprocessing import LabelEncoder

# # Define the model
# model = Sequential([
#     Dense(512, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with input_shape matching your features
#     Dropout(0.5),  # Dropout for regularization
#     Dense(256, activation='relu'),  # Hidden layer
#     Dropout(0.5),  # Dropout for regularization
#     Dense(1, activation='sigmoid')  # Output layer for binary classification
# ])
# model.
#
# # Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
#               metrics=['accuracy'])
#
# # Train the model
# history = model.fit(X_train, y_train,
#                     epochs=10,
#                     batch_size=32,
#                     validation_split=0.2,  # Use a part of the training set for validation
#                     verbose=1)
#
# # Evaluate the model on the test set
# test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
# print('\nTest accuracy:', test_acc)
# print('\nTest loss:', test_loss)


n_features = X_train.shape[1]
# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Recall', 'Precision'])
# fit the model

# use asarray to convert the dataframe to array
X_train_1 = np.asarray(X_train)
y_train_1 = np.asarray(y_train)

model.fit(X_train_1, y_train_1, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
# evaluate the model
X_test_1 = np.asarray(X_test)
y_test_1 = np.asarray(y_test)

loss, accuracy, recall, precision = model.evaluate(X_test_1, y_test_1, verbose=0)
# print the accuracy in a different way
print('Test Accuracy: %.3f' % accuracy)
print('Recall: %.3f' % recall)
print('Precision: %.3f' % precision)




Epoch 1/10
712/712 [==============================] - 2s 2ms/step - loss: 0.2860 - accuracy: 0.9290 - recall: 0.8068 - precision: 0.8485 - val_loss: 0.0935 - val_accuracy: 0.9856 - val_recall: 0.9321 - val_precision: 0.9972
Epoch 2/10
712/712 [==============================] - 1s 2ms/step - loss: 0.0738 - accuracy: 0.9855 - recall: 0.9435 - precision: 0.9869 - val_loss: 0.0653 - val_accuracy: 0.9914 - val_recall: 0.9613 - val_precision: 0.9964
Epoch 3/10
712/712 [==============================] - 1s 2ms/step - loss: 0.0493 - accuracy: 0.9934 - recall: 0.9737 - precision: 0.9947 - val_loss: 0.0434 - val_accuracy: 0.9919 - val_recall: 0.9604 - val_precision: 1.0000
Epoch 4/10
712/712 [==============================] - 1s 2ms/step - loss: 0.0399 - accuracy: 0.9947 - recall: 0.9777 - precision: 0.9970 - val_loss: 0.0507 - val_accuracy: 0.9714 - val_recall: 0.9854 - val_precision: 0.8870
Epoch 5/10
712/712 [==============================] - 1s 2ms/step - loss: 0.0321 - accuracy: 0.9964 - re

In [21]:
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier

# Initialize individual models
log_clf = LogisticRegression(random_state=42, max_iter=1000)
dt_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
#svm_clf = GaussianNB() # Enable probability for soft voting
svm_clf = ExtraTreesClassifier(n_estimators=100, max_depth=10, random_state=42)

# Create ensemble model using soft voting
ensemble_clf = VotingClassifier(estimators=[
    ('lr', log_clf),
    ('dt', dt_clf),
    ('svc', dt_clf),
    ], voting='soft')

ensemble_clf.fit(X_train, y_train)

# Predict on the test set
ensemble_predictions = ensemble_clf.predict(X_test)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
ensemble_CMatrix = confusion_matrix(y_test, ensemble_predictions)

# Print evaluation metrics
print("Ensemble Model Accuracy:", ensemble_accuracy)
print("Ensemble Model Confusion Matrix:", ensemble_CMatrix)
print("Ensemble Classification Report:\n", classification_report(y_test, ensemble_predictions))

import pickle

# Save the ensemble model to disk
with open('ensemble_model.pkl', 'wb') as f:
    pickle.dump(ensemble_clf, f)

print('Ensemble model saved to disk.')

Ensemble Model Accuracy: 0.8313439123718579
Ensemble Model Confusion Matrix: [[5629    0]
 [1201  291]]
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.82      1.00      0.90      5629
           1       1.00      0.20      0.33      1492

    accuracy                           0.83      7121
   macro avg       0.91      0.60      0.62      7121
weighted avg       0.86      0.83      0.78      7121

Ensemble model saved to disk.
